In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

sns.set_style("whitegrid")
sns.set_palette("pastel")

In [2]:
df = pd.read_csv("../data/korean_data.tsv", sep="\t")
df.head()

,Rating,Review
0,1,고기가 정말 맛있었어요! 육즙이 가득 있어서 너무 좋았아요 일하시는 분들 너무 친절...
1,1,"잡내 없고 깔끔, 담백한 맛의 순댓국이 순댓국을 안 좋아하는 사람들에게도 술술 넘어..."
2,1,고기 양이 푸짐해서 특 순대국밥을 시킨 기분이 듭니다 맛도 좋습니다 다만 양념장이 ...
3,1,순댓국 자체는 제가 먹어본 순대국밥집 중에서 Top5 안에는 들어요. 그러나 밥 양...
4,1,괜찮아요. 가격 대비 양 많고... 무난한 식당이었네요. 순댓국은 푸짐한 느낌이고 ...


### Rename Columns

In [3]:
df.rename(columns={"Rating": "Sentiment", "Review": "Comment"}, inplace=True)
df.head()

,Sentiment,Comment
0,1,고기가 정말 맛있었어요! 육즙이 가득 있어서 너무 좋았아요 일하시는 분들 너무 친절...
1,1,"잡내 없고 깔끔, 담백한 맛의 순댓국이 순댓국을 안 좋아하는 사람들에게도 술술 넘어..."
2,1,고기 양이 푸짐해서 특 순대국밥을 시킨 기분이 듭니다 맛도 좋습니다 다만 양념장이 ...
3,1,순댓국 자체는 제가 먹어본 순대국밥집 중에서 Top5 안에는 들어요. 그러나 밥 양...
4,1,괜찮아요. 가격 대비 양 많고... 무난한 식당이었네요. 순댓국은 푸짐한 느낌이고 ...


In [4]:
df.shape

(641761, 2)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 641761 entries, 0 to 641760
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Sentiment  641761 non-null  int64 
 1   Comment    641761 non-null  object
dtypes: int64(1), object(1)
memory usage: 9.8+ MB


In [6]:
df.shape

(641761, 2)

In [7]:
# 0 negative, 1 positive 2 neutral/ambiguous
sentiment_mapping = {0: "negative", 1 : "positive",  2 : "neutral"}
df["Sentiment"] = df["Sentiment"].replace(sentiment_mapping).astype(str)
df["Sentiment"].value_counts()

Sentiment
positive    388110
neutral     182741
negative     70910
Name: count, dtype: int64

In [8]:
df[df["Sentiment"] == "neutral"].head()

,Sentiment,Comment
8,neutral,늦은 밤 찾아간 곳. 사람들로 북적북적. 맛있습니다.
9,neutral,주차된 차도 많고 줄 서서 먹음. 순대는 맛있으나 순댓국에 들어간 고기에서 잡내가 ...
14,neutral,유명세에 비해 순대 맛은 그저 그런 순대 속이 그냥 당면이다
19,neutral,"오래되고 유명하다고 해서 가봤는데, 순댓국 맛은 보통이었어요. 직원분들이 많고 친절..."
20,neutral,"찹쌀순대 좋아해서 가는 곳, 동네에선 예전부터 유명했던 곳이다. 사실 맛이 특별하게..."


In [9]:
## Will not be using neutral reviews
df = df[df["Sentiment"] != "neutral"]

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 459020 entries, 0 to 641758
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Sentiment  459020 non-null  object
 1   Comment    459020 non-null  object
dtypes: object(2)
memory usage: 10.5+ MB


In [11]:
df["Sentiment"].value_counts()

Sentiment
positive    388110
negative     70910
Name: count, dtype: int64

In [436]:
## TODO: IMBALANCED

### Handle null values

In [12]:
df["Comment"] = df["Comment"].str.strip()
null_comments = df[df["Comment"].isnull() | (df["Comment"] == "")]
null_comments.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11 entries, 165766 to 565974
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Sentiment  11 non-null     object
 1   Comment    11 non-null     object
dtypes: object(2)
memory usage: 264.0+ bytes


### Handle duplicates

In [13]:
print(df.duplicated().sum())

1694


In [14]:
df = df.drop_duplicates()
print(df.duplicated().sum())

0


In [15]:
null_comments.head(8)

,Sentiment,Comment
165766,positive,
190263,positive,
191900,negative,
204194,positive,
206087,negative,
208772,positive,
213611,negative,
229888,positive,


In [16]:
df = df[df["Comment"].notna() & (df["Comment"] != "")]
df[df["Comment"].isnull() | (df["Comment"] == "")]

,Sentiment,Comment


In [17]:
df.value_counts("Sentiment")

Sentiment
positive    386583
negative     70741
Name: count, dtype: int64

In [18]:
# Using 50000 rows only
each_class = 50000

df_positive = df[df["Sentiment"] == "positive"].iloc[:each_class]
df_negative = df[df["Sentiment"] == "negative"].iloc[:each_class]

new_df = pd.concat([df_positive, df_negative])
df = new_df.sample(frac=1, random_state=42).reset_index(drop=True)

df["Sentiment"].value_counts()

Sentiment
negative    50000
positive    50000
Name: count, dtype: int64

In [19]:
## Will be using this much only ignore the rest below for now
df.to_csv("../data/korean_only_data.csv", index=False)

### Another dataset
- https://github.com/bab2min/corpus/tree/master/sentiment

In [444]:
df_another = pd.read_csv(
    "../data/naver_shopping.txt",
    sep="\t",
    header=None,
    names=["Sentiment", "Comment"],
)
df_another.head()

,Sentiment,Comment
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


In [445]:
df_another.value_counts("Sentiment")

Sentiment
5    81177
2    63989
1    36048
4    18786
Name: count, dtype: int64

In [446]:
df_another = df_another[df_another["Sentiment"].isin([1, 2])]

In [447]:
df_another.value_counts("Sentiment")

Sentiment
2    63989
1    36048
Name: count, dtype: int64

In [448]:
df_another["Sentiment"] = df_another["Sentiment"].replace(
    {1: "negative", 2: "negative"}
)

In [449]:
df_another["Sentiment"].value_counts()

Sentiment
negative    100037
Name: count, dtype: int64

In [450]:
df_another["Comment"] = df_another["Comment"].str.strip()
null_comments = df_another[
    df_another["Comment"].isnull() | (df_another["Comment"] == "")
]
null_comments.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Sentiment  0 non-null      object
 1   Comment    0 non-null      object
dtypes: object(2)
memory usage: 0.0+ bytes


In [451]:
print(df_another.duplicated().sum())

77


In [452]:
df_another.drop_duplicates(inplace=True)
print(df_another.duplicated().sum())

0


In [453]:
df_another.value_counts("Sentiment")

Sentiment
negative    99960
Name: count, dtype: int64

In [454]:
print(df.duplicated().sum())
print(df_another.duplicated().sum())

0
0


In [455]:
concat_df = pd.concat([df, df_another])
final_df = concat_df.sample(frac=1, random_state=42).reset_index(drop=True)

final_df["Sentiment"].value_counts()

Sentiment
positive    220000
negative    170701
Name: count, dtype: int64

In [456]:
final_df.duplicated().sum()

np.int64(27)

In [457]:
final_df.drop_duplicates(inplace=True)
final_df.duplicated().sum()

np.int64(0)

In [458]:
final_df["Sentiment"].value_counts()

Sentiment
positive    220000
negative    170674
Name: count, dtype: int64

## Another Dataset
- https://github.com/bab2min/corpus/tree/master/sentiment

In [459]:
third_df = pd.read_csv(
    "../data/steam.txt",
    sep="\t",
    header=None,
    names=["Sentiment", "Comment"],
)
third_df.head()

,Sentiment,Comment
0,0,노래가 너무 적음
1,0,"돌겠네 진짜. 황숙아, 어크 공장 그만 돌려라. 죽는다."
2,1,막노동 체험판 막노동 하는사람인데 장비를 내가 사야돼 뭐지
3,1,차악!차악!!차악!!! 정말 이래서 왕국을 되찾을 수 있는거야??
4,1,시간 때우기에 좋음.. 도전과제는 50시간이면 다 깰 수 있어요


In [460]:
third_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Sentiment  100000 non-null  int64 
 1   Comment    100000 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.5+ MB


In [461]:
third_df.value_counts("Sentiment")

Sentiment
0    50004
1    49996
Name: count, dtype: int64

In [462]:
third_df = third_df[third_df["Sentiment"].isin([0])]

In [463]:
third_df["Sentiment"] = third_df["Sentiment"].replace({0: "negative"})

In [464]:
third_df.value_counts("Sentiment")

Sentiment
negative    50004
Name: count, dtype: int64

In [465]:
df_another["Comment"] = df_another["Comment"].str.strip()
null_comments = df_another[
    df_another["Comment"].isnull() | (df_another["Comment"] == "")
]
null_comments.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Sentiment  0 non-null      object
 1   Comment    0 non-null      object
dtypes: object(2)
memory usage: 0.0+ bytes


In [466]:
print(df_another.duplicated().sum())

0


In [467]:
final_df["Sentiment"].value_counts()

Sentiment
positive    220000
negative    170674
Name: count, dtype: int64

In [468]:
remaining_rows = 220000 - 170674
remaining_rows

49326

In [469]:
balanced_df = pd.concat([final_df, third_df.iloc[:remaining_rows + 48]])
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

balanced_df["Sentiment"].value_counts()

Sentiment
negative    220048
positive    220000
Name: count, dtype: int64

In [470]:
balanced_df.duplicated().sum()

np.int64(48)

In [471]:
balanced_df.drop_duplicates(inplace=True)
balanced_df.duplicated().sum()

np.int64(0)

In [472]:
balanced_df["Sentiment"].value_counts()

Sentiment
negative    220000
positive    220000
Name: count, dtype: int64

In [473]:
balanced_df.to_csv("../data/korean_only_data.csv", index=False)